In [ ]:
!pip install accelerate

In [ ]:
import accelerate
print(accelerate.__version__)

In [ ]:
import pandas as pd
import numpy as np

def get_promts(seed1, seed2, seed3, seed0, promt_shuffle_type, num_prompt_examples):
  df_raw_processed = pd.read_csv('raw_processed.csv')

  np.random.seed(seed1)
  random_numbers_positive = np.random.randint(0, 2500000/2, size=60)
  np.random.seed(seed2)
  random_numbers_negative = np.random.randint(2500000/2, 2500000, size=60)

  prompt_task = "Predict the emotion of a tweet. Answer with a single word: positive or negative\n"
  ## prompt_task = "What is the emotion of the following tweet? Please answer with a single word: positive or negative\n"
  ## prompt_examples = prompt_task + "Examples:\n"
  prompt_examples = "Examples of how to predict the emotion of tweets:\n"
  count_positive = 0
  count_negative = 0
  array = None
  if promt_shuffle_type == 0:
    array = np.array([0]*num_prompt_examples + [1]*num_prompt_examples)
  if promt_shuffle_type == 1 or promt_shuffle_type ==2:
    array = np.array([1]*num_prompt_examples + [0]*num_prompt_examples)
  if promt_shuffle_type == 2:
    np.random.seed(seed3)
    np.random.shuffle(array)
  for i in range(len(array)):
    if array[i]==1:
      prompt_examples = prompt_examples + "{}) Tweet: ".format(i+1) + str(df_raw_processed.iloc[random_numbers_positive[count_positive]]['text']) + "\nAnswer: positive\n"
      count_positive += 1
    else:
      prompt_examples = prompt_examples + "{}) Tweet: ".format(i+1) + str(df_raw_processed.iloc[random_numbers_negative[count_negative]]['text']) + "\nAnswer: negative\n"
      count_negative += 1

  prompt_examples += "Predict the sentiment of the following tweet. Answer with only a single word: positive or negative\n"
  df_raw_processed = pd.read_csv('raw_processed.csv')
  prompts = []
  prompts_ground_truth = []
  count = 0
  np.random.seed(seed0)
  random_numbers = np.random.randint(0, 2500000, size=1220)
  for i in range(1220):
    if count == 100:
      break
    if random_numbers[i] in random_numbers_positive or random_numbers[i] in random_numbers_negative:
      continue
    new_prompt = prompt_examples + "Tweet: " + str(df_raw_processed.iloc[random_numbers[i]]['text']) + "\nAnswer: "
    prompts.append(new_prompt)
    prompts_ground_truth.append((lambda label: 'positive' if label == 1 else 'negative')(df_raw_processed.iloc[random_numbers[i]]['labels']))
    count += 1

  return prompts, prompts_ground_truth

In [ ]:
prompts_list = []
number_of_examples = 11
prompts1, prompts_ground_truth = get_promts(85653, 8551, 8752, 42, 1, number_of_examples)
prompts2, prompts_ground_truth = get_promts(5577, 325, 9843, 42, 2, number_of_examples)
prompts3, prompts_ground_truth = get_promts(32329, 12843, 6326, 42, 2, number_of_examples)
prompts4, prompts_ground_truth = get_promts(5910, 2665, 1234, 42, 2, number_of_examples)
prompts5, prompts_ground_truth = get_promts(13947, 54573, 2024, 42, 2, number_of_examples)
prompts6, prompts_ground_truth = get_promts(357, 9827, 1391, 42, 2, number_of_examples)
prompts7, prompts_ground_truth = get_promts(171, 43699, 2875, 42, 2, number_of_examples)
prompts8, prompts_ground_truth = get_promts(253, 853, 123592, 42, 2, number_of_examples)
prompts_list.append(prompts1)
prompts_list.append(prompts2)
prompts_list.append(prompts3)
prompts_list.append(prompts4)
prompts_list.append(prompts5)
prompts_list.append(prompts6)
prompts_list.append(prompts7)
prompts_list.append(prompts8)

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import accelerate
device = "cuda"

model = AutoModelForCausalLM.from_pretrained("Qwen/Qwen1.5-7B-Chat", device_map="auto")
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen1.5-7B-Chat")

prompt = prompts1[0]

prompts_collection = [{"role": "user", "content": prompt}]

text = tokenizer.apply_chat_template(prompts_collection, tokenize=False, add_generation_prompt=True)

samples = tokenizer([text], return_tensors="pt").to(device)

samples_generated = model.generate(samples.input_ids, max_new_tokens=512, do_sample=True)

samples_generated = [output_ids[len(input_ids):] for input_ids, output_ids in zip(samples.input_ids, samples_generated)]

response = tokenizer.batch_decode(samples_generated, skip_special_tokens=True)[0]
print(response)

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
import torch.nn.functional as F
responses = []
number_of_responses = len(prompts1)
count_correct = 0

for i, prompt in enumerate(prompts1):
  if (i % 10 == 0):
    print(i)
  prompts_collection = [{"role": "user", "content": prompt}]

  text = tokenizer.apply_chat_template(prompts_collection, tokenize=False, add_generation_prompt=True)
  samples = tokenizer([text], return_tensors="pt", padding=True)

  samples = {k: v.to(device) for k, v in samples.items()}

  with torch.no_grad():
      outputs = model.generate(samples["input_ids"],
                              attention_mask=samples["attention_mask"],
                              max_new_tokens=5,
                              do_sample=True,
                              return_dict_in_generate=True,
                              output_scores=False)



  samples_generated = [output_ids[len(input_ids):] for input_ids, output_ids in zip(samples["input_ids"], outputs.sequences)]
  response = tokenizer.batch_decode(samples_generated, skip_special_tokens=True)[0]

  response = response.replace('.', '').lower()

  if response == prompts_ground_truth[i]:
      count_correct += 1

  responses.append(response)
  del samples
  torch.cuda.empty_cache()
print("Accuracy: {:.4f}%".format(float(count_correct / number_of_responses)))



Disclaimer:
Code contains refactored and reused code from:


*   https://huggingface.co/Qwen/Qwen1.5-7B-Chat


*   https://qwenlm.github.io/blog/qwen1.5/
*   https://github.com/QwenLM/Qwen2


